# 实际部署：SO-101 5-DOF 双臂 IK求解演示

## 1. 初始化

In [ ]:
import time
import numpy as np
from scipy.spatial.transform import Rotation as R
from driver.ftservo_controller import ServoController
from ik.robot import create_so101_5dof
from hw_solver_wrapper import create_hw_solver

controller = ServoController(port="/dev/right_arm", baudrate=1_000_000, config_path="./driver/right_arm.json")
robot = create_so101_5dof()
robot.set_servo_controller(controller)
print("初始化硬件求解器...")

hw_solver = create_hw_solver(robot, bitstream_path='design_1.bit')

if hw_solver and hw_solver.hw_available:
    print("✓ 硬件求解器就绪\n")
else:
    print("⚠ 硬件求解器初始化失败或不可用\n")

q0 = np.zeros(5)
controller.move_all_home()
time.sleep(1)

q0 = robot.read_joint_angles(joint_names=robot.joint_names, verbose=True)
T_current = robot.fkine(q0)
print("\n🔍 当前末端位姿矩阵：")
print(np.round(T_current, 3))
euler = R.from_matrix(T_current[:3, :3]).as_euler('xyz', degrees=False)
print(f"当前位置: x={T_current[0,3]:.4f}, y={T_current[1,3]:.4f}, z={T_current[2,3]:.4f}")
print(f"当前姿态: roll={euler[0]:.4f}, pitch={euler[1]:.4f}, yaw={euler[2]:.4f}")


✅ 已加载舵机配置:
  1: shoulder_pan (home_pose=1974, range=797~3151)
  2: shoulder_lift (home_pose=2130, range=946~3314)
  3: elbow_flex (home_pose=1906, range=803~3009)
  4: wrist_flex (home_pose=2043, range=870~3216)
  5: wrist_roll (home_pose=2032, range=120~3945)
  6: gripper (home_pose=1993, range=1993~3507)
初始化硬件求解器...
✓ 硬件求解器已初始化 (基址: 0x40000000, 核名: xf_solver_lm_so101_0)
✓ 硬件求解器就绪

🏠 全部舵机同步回中位完成

📡 读取关节角度:
 shoulder_pan    : 步数=1977, Δ=   +3 → q=-0.0046 rad 
 shoulder_lift   : 步数=2156, Δ=  +26 → q=+0.0399 rad 
 elbow_flex      : 步数=1930, Δ=  +24 → q=+0.0368 rad 
 wrist_flex      : 步数=2044, Δ=   +1 → q=-0.0015 rad 
 wrist_roll      : 步数=2035, Δ=   +3 → q=+0.0046 rad 

🔍 当前末端位姿矩阵：
[[ 0.997  0.005  0.075  0.338]
 [-0.005  1.    -0.005 -0.001]
 [-0.075  0.005  0.997  0.225]
 [ 0.     0.     0.     1.   ]]
当前位置: x=0.3379, y=-0.0013, z=0.2251
当前姿态: roll=0.0046, pitch=0.0752, yaw=-0.0046


## 2. 硬件求解器定义

## 3. 硬件求解IK

In [9]:
def build_target_pose(x=0.5, y=0, z=0.1, roll=0.0, pitch=np.pi/4, yaw=0.0):
    r = R.from_euler('xyz', [roll, pitch, yaw], degrees=False)
    T = np.eye(4)
    T[:3, :3] = r.as_matrix()
    T[:3, 3] = [x, y, z]
    return T

T_goal = build_target_pose(x=0.3, y=-0.2, z=0.1, roll=-np.pi/4, pitch=np.pi/6, yaw=0)
print("\n🎯 目标末端位姿矩阵：")
print(np.round(T_goal, 3))
print(f"目标位置: x={T_goal[0,3]:.4f}, y={T_goal[1,3]:.4f}, z={T_goal[2,3]:.4f}")

mask = [1, 1, 1, 0.8, 0.8, 0]

# 优先使用硬件求解，如果不可用则使用纯 Python (新逻辑)
if hw_solver and hw_solver.hw_available:
    print("\n🔄 开始硬件加速求解 (Method: Sugihara)...")
    sol = hw_solver.solve(
        T_goal=T_goal,
        q0=q0,
        ilimit=500,
        slimit=50,
        tol=1e-7,
        mask=mask,
        k=0.1,
        method="sugihara"  # 使用 Sugihara 方法以提高收敛性
    )
else:
    print("\n⚠ 硬件不可用，使用纯Python求解 (Method: Sugihara)...")
    sol = robot.ikine_LM(
        Tep=T_goal,
        q0=q0,
        ilimit=500,
        slimit=50,
        tol=1e-3,
        mask=mask,
        k=0.1,
        method="sugihara"
    )

if not sol.success:
    print("\n❌ 求解失败")
    print(f"原因: {sol.reason}")
    # controller.close() # 保持连接以便调试
else:
    print("\n✅ 求解成功")
    print(f"求解耗时: {sol.total_time*1000:.2f} ms")
    print(f"迭代次数: {sol.iterations}")
    print(f"目标关节角(rad): {np.round(sol.q, 4)}")
    
    T_tar = robot.fkine(sol.q)
    pos_error = np.linalg.norm(T_tar[:3,3] - T_goal[:3,3])
    print(f"末端位置误差: {pos_error*1000:.2f} mm")


🎯 目标末端位姿矩阵：
[[ 0.866 -0.354  0.354  0.3  ]
 [ 0.     0.707  0.707 -0.2  ]
 [-0.5   -0.612  0.612  0.1  ]
 [ 0.     0.     0.     1.   ]]
目标位置: x=0.3000, y=-0.2000, z=0.1000

🔄 开始硬件加速求解 (Method: Sugihara)...

[硬件求解器] 启动逆运动学求解
  初始关节角: [-0.00460194  0.0398835   0.03681554 -0.00153398  0.00460194]
  最大迭代数: 500
  收敛阈值: 1e-07, 初始阻尼: 0.1, 方法: sugihara
  硬件加速: ✓ 可用
  迭代    0: 误差=8.150063e-01, E=3.321176e-01, λ_eff=4.3212e-01
  迭代    1: 误差=3.762041e-01, E=7.076478e-02, λ_eff=1.7076e-01
  迭代    2: 误差=1.885168e-01, E=1.776930e-02, λ_eff=1.1777e-01
  迭代  100: 误差=8.915100e-04, E=3.973951e-07, λ_eff=1.0000e-01

✓ 收敛成功 (迭代 120, 耗时 0.6284s)

✅ 求解成功
求解耗时: 628.38 ms
迭代次数: 120
目标关节角(rad): [-0.6972  0.6053 -0.0996  0.0179 -0.7854]
末端位置误差: 0.44 mm


## 4. 执行运动

In [10]:
home_pose = {}
for name in robot.joint_names:
    home_pose[name] = controller.get_home_position(name)

servo_targets = robot.q_to_servo_targets(q_rad=sol.q, home_pose=home_pose)

for k in list(servo_targets.keys()):
    servo_targets[k] = controller.limit_position(k, servo_targets[k])

print("\n📋 目标舵机步数：")
for k in robot.joint_names:
    q0_servo = robot.q_to_servo_targets(q0, home_pose=home_pose)[k]
    delta = servo_targets[k] - q0_servo
    print(f"  {k}: {servo_targets[k]} (Δ={delta:+d})")

input("\n按 Enter 执行...")
controller.soft_move_to_pose(servo_targets, step_count=5, interval=0.08)
time.sleep(1)


📋 目标舵机步数：
  shoulder_pan: 2428 (Δ=+451)
  shoulder_lift: 2525 (Δ=+369)
  elbow_flex: 1841 (Δ=-89)
  wrist_flex: 2031 (Δ=-13)
  wrist_roll: 1520 (Δ=-515)

按 Enter 执行...
🌀 开始软启动移动到目标姿态...
  Step 1/5
  Step 2/5
  Step 3/5
  Step 4/5
  Step 5/5
✅ 目标姿态已平滑到位


## 5. 验证结果

In [11]:
q_actual = robot.read_joint_angles(joint_names=robot.joint_names, verbose=True)
T_actual = robot.fkine(q_actual)
print("\n🔍 执行后末端位姿：")
print(np.round(T_actual, 3))

actual_error = np.linalg.norm(T_actual[:3,3] - T_goal[:3,3])
print(f"位置误差: {actual_error*1000:.2f} mm")

print("\n✅ 完成")
controller.close()


📡 读取关节角度:
 shoulder_pan    : 步数=2424, Δ= +450 → q=-0.6903 rad 
 shoulder_lift   : 步数=2539, Δ= +409 → q=+0.6274 rad 
 elbow_flex      : 步数=1865, Δ=  -41 → q=-0.0629 rad 
 wrist_flex      : 步数=2043, Δ=   +0 → q=+0.0000 rad 
 wrist_roll      : 步数=1521, Δ= -511 → q=-0.7839 rad 

🔍 执行后末端位姿：
[[ 0.651  0.16   0.742  0.298]
 [-0.538  0.787  0.303 -0.196]
 [-0.535 -0.596  0.598  0.088]
 [ 0.     0.     0.     1.   ]]
位置误差: 13.05 mm

✅ 完成
